# [E-04]Lyricist ( 작사가 )
_____
# 목차
## 1. 개요
    1.1 들어가기에 앞서
    1.2 루브릭 평가기준
## 2. 프로젝트: 멋진 작사가 만들기
    2.1 데이터 읽어오기
    2.2 데이터 정제
    2.3 데이터셋 분리
    2.4 모델 학습 및 평가
## 3. 결론
    3.1 참조
    3.2 결론
   
    
---


## 1. 개요
### 1.1 들어가기에 앞서

#### 이론

##### GPT-2
- 작문할 수 있는 딥러닝 모델을 말한다.

##### 언어모델
- n−1개의 단어 시퀀스 w{1}, ... , w{n-1}가 주어졌을 때, n번째 단어 w{n}으로 무엇이 올지 예측하는 확률 모델

##### 토큰화(Tokenize)
- 문장을 일정한 기준으로 쪼개는 과정

##### 소스 문장(Source Sentence)
- 자연어처리 분야에서 모델의 입력이 되는 문장(x_train)

##### 타겟 문장(Target Sentence)
- 정답 역할을 하게 될 모델의 출력 문장(y_train)

##### 벡터화(vectorize)
- tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 번에 해준다

##### 텐서(tensor)
- 숫자로 변환된 데이터

##### Embedding
- 단어를 추상적으로 변환하는 역할

##### RNN1(LSTM),RNN2(LSTM) 
- 문장을 순차적으로 읽으며 단어 간의 연관성을 분석

##### Linear(Dense)
- RNN에서 만들어낸 결과를 바탕으로 생성할 단어를 결정

----

### 1.2 루브릭 평가기준

평가문항|상세기준
-|-
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?|텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?|특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

----



## 2. 프로젝트: 멋진 작사가 만들기
### 2.1 데이터 읽어오기

In [1]:
# 필요한 모듈 import 하기

import glob
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


---

### 2.2 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


1. sentence = sentence.lower().strip()
- 소문자로 바꾸고, 양쪽 공백을 지운다.

2. sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
- 특수문자 양쪽에 공백을 넣는다.

3. sentence = re.sub(r'[" "]+', " ", sentence)
- 여러개의 공백은 하나의 공백으로 바꾼다.

4. sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
- a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다

5. sentence = sentence.strip()
- 양쪽 공백을 지운다.

6. sentence = '<start> ' + sentence + ' <end>'
- 문장 시작에는 <start>, 끝에는 <end>를 추가한다.

In [4]:
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    word_list = preprocessed_sentence.split() 
    if len(word_list) > 15: continue # 토큰의 개수 15개 초과하면 제외
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 10개 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 위해
- if len(word_list) > 15: continue를 사용

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들기
    # 우리는 이미 문장을 정제했으니 filters가 필요없음
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞추기
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f249c5650a0>


In [6]:
print(tensor[:3, :10])

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성

# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


---

### 2.3 데이터셋 분리

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만들기

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

- enc_train, dec_train 를 하나로 묶어 dataset으로 만든다.
- enc_val, dec_val 를 하나로 묶어 val_dataset으로 만든다

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512 # embedding_size: 워드 벡터의 차원수
hidden_size = 2048 # hidden_size: LSTM 레이어의 hidden state 의 차원수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- 하이퍼파라미터인 embedding_size와 hidden_size를 조절하며 loss와 val_loss값 조정

In [12]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 8.11766367e-05, -1.47675528e-04,  1.69841675e-04, ...,
          1.88826991e-04,  1.91001673e-05,  2.40910122e-05],
        [ 2.99572595e-04, -7.05683415e-05,  3.05791269e-04, ...,
          1.66958576e-04, -6.74303228e-05,  1.60884782e-04],
        [ 2.09231934e-04, -4.38166375e-04,  5.16903063e-04, ...,
          2.56792875e-04, -1.67655569e-04,  4.67032485e-04],
        ...,
        [-1.86625705e-03, -2.40157358e-03,  7.79911585e-04, ...,
         -1.08572864e-03, -9.94169386e-04,  9.37105273e-04],
        [-1.90591579e-03, -2.79953983e-03,  4.18997108e-04, ...,
         -1.05133140e-03, -8.44379014e-04,  1.44546584e-03],
        [-1.92077784e-03, -3.17416457e-03,  4.69305669e-05, ...,
         -9.74238792e-04, -6.88639295e-04,  1.96792139e-03]],

       [[ 8.11766367e-05, -1.47675528e-04,  1.69841675e-04, ...,
          1.88826991e-04,  1.91001673e-05,  2.40910122e-05],
        [ 2.29892001e-04, -1.74049695e-04,  3

---

### 2.4 모델 학습 및 평가

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=val_dataset, verbose=1)

Epoch 1/10
487/487 [==============================] - 240s 488ms/step - loss: 3.2881 - val_loss: 2.9086
Epoch 2/10
487/487 [==============================] - 237s 486ms/step - loss: 2.7364 - val_loss: 2.6596
Epoch 3/10
487/487 [==============================] - 237s 487ms/step - loss: 2.4250 - val_loss: 2.4848
Epoch 4/10
487/487 [==============================] - 238s 488ms/step - loss: 2.1188 - val_loss: 2.3494
Epoch 5/10
487/487 [==============================] - 238s 488ms/step - loss: 1.8304 - val_loss: 2.2514
Epoch 6/10
487/487 [==============================] - 238s 488ms/step - loss: 1.5740 - val_loss: 2.1896
Epoch 7/10
487/487 [==============================] - 237s 487ms/step - loss: 1.3611 - val_loss: 2.1538
Epoch 8/10
487/487 [==============================] - 238s 488ms/step - loss: 1.1982 - val_loss: 2.1487
Epoch 9/10
487/487 [==============================] - 238s 489ms/step - loss: 1.0883 - val_loss: 2.1634
Epoch 10/10
487/487 [==============================] - 238s 488m

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만든다.
    #    1. 입력받은 문장의 텐서를 입력
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑는다.
    #    3. 2에서 예측된 word index를 문장 뒤에 붙인다.
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so , <end> '

---

## 3. 결론
### 3.1 참조
- 글자 수 이상문장제외: https://www.kite.com/python/answers/how-to-count-the-number-of-words-in-a-string-in-python
- val_loss : https://github.com/JaeHeee/AIFFEL_Project/blob/master/EXPLORATION/EXPLORATION%2011.%20%EC%9E%91%EC%82%AC%EA%B0%80%20%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%20%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb

---

### 3.2 회고
- 프로젝트 노드에서 '토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기'라는 내용을 요구하는 문항이 있었다. 이 문항에 대해 어떤 코드를 작성해야 할 지, 또한 요구하는 게 무엇인지 파악이 불가하여 조원 수진님을 통해 참조하셨던 링크를 받아 if 조건문에 continue를 사용하여 코드 작성을 했다. 

- 프로젝트에서 요구하는 val_loss 값은 2.2였는데 요구하는 값에 딱 맞추기에는 너무나 힘든 과정이었다. embedding_size와 hidden_size를 조절하여 val_loss 값을 낮춰나갔는데, 이 값들의 크기를 높게 조절할 수록 학습 시간은 점점 더 길어졌었다. 중간에 조원중 진환님께서 하이퍼 파라미터를 어떻게 조절하는 게 좋을 지에 대해 조언을 해주셔서 다른 사람들에 비해 비교적 쉽게 val_loss 값을 낮출 수 있었다.

- 결론에 대해 짚으면, 마지막에 모델을 통해 생성한 문장의 결과가 찝찝하게 나온 것 같다. 완벽한 문장이 아니라 'i love you so,' 에서 끊겼기 때문에 답답한 심정이다. 원인을 찾지 못했기에 일단은 저 결과에 만족할 수 밖에 없었던 것 같다.

- 이번 프로젝트에 대해 요지 자체를 파악하지 못한 것 같아 아쉬움이 많았다. 그만큼 자연어 처리가 어렵고 조금 난해한 분야라는 느낌을 받았다. 아직은 나에게 맞는 분야를 찾아가는 단계이기에 다음 NLP노드에서는 자연어처리의 색 다른 매력이 있을 지 노드를 좀 더 꼼꼼하게 살펴보도록 해야겠다.